# setup and load data dir

* **TODO**: modify **data_dir**, **model_dir** according to your setup
* modify **train.txt** and **test.txt** according to your needs, you may find **all.txt** useful as it records all files


In [1]:
import os
import numpy as np

data_dir = "E:\\cs341-project\\iotdata\\npz_data_all"
model_dir = "E:\\cs341-project\\models"
good_model_dir = "E:\\cs341-project\\models\\good_models"
code_dir = os.getcwd()

# previously train.txt and test.txt are in data_dir, comment next line if they are in code_dir, uncomment if they are in data_dir
# os.chdir(data_dir)

train_f = []
test_f = []
with open("train.txt") as train_fnames:
    for line in train_fnames:
        train_f.append(line.split(" ")[0])
with open("test.txt") as test_fnames:
    for line in test_fnames:
        test_f.append(line.split(" ")[0])

# Model

In [2]:
os.chdir(code_dir)

from util import Heater_Data, gen_sim_data, Solver
from basic_lstm import Basic_LSTM
from lstm_cnn import LSTM_CNN
from bidir_lstm import BiLSTM
from lstm_cnn_deep import DeepCNN_LSTM
from lstm_cnn_deeper import DeeperCNN_LSTM

# setup model
lstm_model = Basic_LSTM(num_units = 80, learning_rate = 0.001, dropout_rate = 0.5, debug=False)
lstm_cnn_model = LSTM_CNN(num_units = 100, learning_rate = 0.001, dropout_rate = 0.5, cnn_args = {'filter_size': 30, 'filter_num': 48}, debug=False)
bilstm_model = BiLSTM(learning_rate = 0.001, dropout_rate = 0.5, debug=False)
deep_cnn_lstm_model = DeepCNN_LSTM(dropout_rate = 0, reg_constant = 0.01)
deeper_cnn_lstm_model = DeeperCNN_LSTM(dropout_rate = 0, reg_constant = 0.01)

# select which one to use
model = deep_cnn_lstm_model

## Test on simulated data
* notice that sim data has different dimension from real data: it might cause some error if you didn't program smart enough

In [3]:
model = deep_cnn_lstm_model
model.initial()

X, y, X_, y_ = gen_sim_data()
for _ in range(10):
#     print(X.shape)
    X_train = np.concatenate([X_, X])
    y_train = np.concatenate([y_, y])
    
    it, loss = model.fit(X_train, y_train)
    print(it, loss)

loss, pred = model.predict(X_, y_)
print("loss:", loss, "acc:", np.mean(pred==y_))
loss, pred = model.predict(X, y)
print("loss:", loss, "acc:", np.mean(pred==y))

model.finish()

1 1.48066
2 1.48832
3 1.21588
4 1.05367
5 0.994193
6 0.859406
7 0.779886
8 0.743629
9 0.722265
10 0.662694
loss: 0.636119 acc: 1.0
loss: 0.630858 acc: 1.0


## Test on one heater
* The goal is to make sure that our model can overfit on train/test on the same heater

### load data into memory

In [3]:
# data config
D_config = {"batch_size":50, "shuffle":True, "balance":{0:1, 1:1, "min_n":60}, "padding":False, "free_mem":False}
os.chdir(data_dir)

for one_file in os.listdir(data_dir):
    if one_file.endswith(".npz"):
        if one_file.split(".")[-2] in train_f:
            D = Heater_Data(one_file)
            break
            
D.config(**D_config)
D.load_into_mem()
print("data loaded into memory")

data loaded into memory


### trainning and testing on one heater

In [4]:
max_loop = 10
model.initial()
# if you want to use previously saved model, comment previous line and uncomment the next line
# model.initial(os.path.join(model_dir, "saved.model"))

for m in range(max_loop):
    d = np.array(D.get_batch())
    y = (d[:, 0, 0] > 0).astype(np.int32)
    x = d[:, :, 1:]
#     print("XXXXX", x, "YYYYY", y)
    it, loss = model.fit(x, y)
    print(it, loss, d.shape)
    loss, pred = model.predict(x, y)
    print("batch loss:", loss, "batch acc:", np.mean(pred==y))

# save model
model.save_model(os.path.join(model_dir, "saved.model"))
model.finish()

1 1.40016 (50, 8640, 20)
batch loss: 1.1392 batch acc: 0.82
2 1.00091 (10, 8640, 20)
batch loss: 0.977036 batch acc: 0.9
3 1.2039 (50, 8640, 20)
batch loss: 1.04027 batch acc: 0.82
4 0.893425 (10, 8640, 20)
batch loss: 0.853214 batch acc: 0.9
5 0.937715 (50, 8640, 20)
batch loss: 0.895822 batch acc: 0.98
6 0.859777 (10, 8640, 20)
batch loss: 0.798338 batch acc: 1.0
7 0.833985 (50, 8640, 20)
batch loss: 0.761547 batch acc: 1.0


KeyboardInterrupt: 

### test on the test set
* once again prove that overfitted model won't work on test set
* may be useful for tuning the normalization part

In [ ]:
model.initial(os.path.join(model_dir, "saved.model"))
D_config = {"batch_size":50, "shuffle":True, "balance":{0:1, 1:1, "min_n":60}, "padding":False, "free_mem":True}

for one_file in os.listdir(data_dir):
    if one_file.endswith(".npz"):
        if one_file.split(".")[-2] in test_f:
            D = Heater_Data(one_file)
            D.config(**D_config)
            D.load_into_mem()
            while True:
                d = np.array(D.get_batch())
                if len(d) == 0:
                    break
                y = (d[:, 0, 0] > 0).astype(np.int32)
                x = d[:, :, 1:]
                loss, pred = model.predict(x, y)
                print("batch loss:", loss, "batch acc:", np.mean(pred==y))
                
                # Notice that predict_prob is doing repeated work, here is only for testing and debuging
                # when running on real data, it should be either predict() or predict_prob() but not both
                loss, pred = model.predict_prob(x, y)
                print(loss, pred)

model.finish()
print("prediction done")

# Work on real data!
files/heaters used for training are stored in **train.txt**, and are stored in **test.txt** for testing

**This section is for developing only, real train/test please refer to train.py and test.py**
## train
* Use async multi-threading to load data
* you may need to change **num_file_in_mem**, **num_threads** in the Solver signature:
 set **num_file_in_mem** according to your memory size and set **num_threads** to the number of threads that you want the program use,
 set **None** to **num_threads** will use all cores on your PC.

In [3]:
# data config
D_config = {"batch_size":50, "shuffle":True, "balance":{0:1, 1:1, "min_n":15}, "padding":False, "free_mem":True}
os.chdir(data_dir)

train_cfg = {
    "model":model,
    "max_iter":3000,
    "data_dir":data_dir,
    "fnames":train_f,
    "D_config":D_config,
    "model_save_dir":model_dir,
    "save_every_iter":200,
    "full_batch":True
}

model.initial()
# model.initial(os.path.join(model_dir, "Basic_LSTM-1400.model"))
m_solver = Solver(num_file_in_mem=10, num_threads=None) # num_threads=None means to use all cores
m_solver(**train_cfg)
model.finish()

[INFO] Notice that we are using multiprocessing to load files, so that child processes won't print out on ipython-notebook, which only monitor the parent process. Please check the terminal for more logging info.
[11.79s] iter: 1, loss: 1.4517, input dim: (50, 8640, 20)
[7.69s] iter: 2, loss: 1.4346, input dim: (50, 8640, 20)
[6.67s] iter: 3, loss: 1.3736, input dim: (50, 8640, 20)
[6.42s] iter: 4, loss: 1.3268, input dim: (50, 8640, 20)
[7.52s] iter: 5, loss: 1.2697, input dim: (50, 8640, 20)
[7.67s] iter: 6, loss: 1.2317, input dim: (50, 8640, 20)
[7.95s] iter: 7, loss: 1.1973, input dim: (50, 8640, 20)
[8.12s] iter: 8, loss: 1.0594, input dim: (50, 8640, 20)
[7.93s] iter: 9, loss: 1.1757, input dim: (50, 8640, 20)
[8.76s] iter: 10, loss: 1.4474, input dim: (50, 8640, 20)
[9.11s] iter: 11, loss: 0.9782, input dim: (50, 8640, 20)
[10.21s] iter: 12, loss: 1.0869, input dim: (50, 8640, 20)
[7.83s] iter: 13, loss: 1.1262, input dim: (50, 8640, 20)
[7.74s] iter: 14, loss: 1.1750, input dim

[6.82s] iter: 138, loss: 0.6457, input dim: (50, 8640, 20)
[6.71s] iter: 139, loss: 0.6349, input dim: (50, 8640, 20)
[7.36s] iter: 140, loss: 0.7001, input dim: (50, 8640, 20)
[5.87s] iter: 141, loss: 0.7356, input dim: (50, 8640, 20)
[7.95s] iter: 142, loss: 0.6663, input dim: (50, 8640, 20)
[7.30s] iter: 143, loss: 0.7043, input dim: (50, 8640, 20)
[9.17s] iter: 144, loss: 0.6569, input dim: (50, 8640, 20)
[9.94s] iter: 145, loss: 0.6497, input dim: (50, 8640, 20)
[8.44s] iter: 146, loss: 0.6593, input dim: (50, 8640, 20)
[7.72s] iter: 147, loss: 0.6928, input dim: (50, 8640, 20)
[9.00s] iter: 148, loss: 0.5765, input dim: (50, 8640, 20)
[7.53s] iter: 149, loss: 0.8079, input dim: (50, 8640, 20)
[7.99s] iter: 150, loss: 0.8015, input dim: (50, 8640, 20)
[7.34s] iter: 151, loss: 0.7541, input dim: (50, 8640, 20)
[7.13s] iter: 152, loss: 0.6956, input dim: (50, 8640, 20)
[7.29s] iter: 153, loss: 0.6012, input dim: (50, 8640, 20)
[8.33s] iter: 154, loss: 0.6236, input dim: (50, 8640, 2

[7.36s] iter: 276, loss: 0.7167, input dim: (50, 8640, 20)
[7.45s] iter: 277, loss: 0.6274, input dim: (50, 8640, 20)
[7.28s] iter: 278, loss: 0.5027, input dim: (50, 8640, 20)
[8.13s] iter: 279, loss: 0.4926, input dim: (50, 8640, 20)
[8.21s] iter: 280, loss: 0.6225, input dim: (50, 8640, 20)
[9.32s] iter: 281, loss: 0.6925, input dim: (50, 8640, 20)
[9.47s] iter: 282, loss: 0.5762, input dim: (50, 8640, 20)
[6.12s] iter: 283, loss: 0.6260, input dim: (50, 8640, 20)
[7.93s] iter: 284, loss: 0.6643, input dim: (50, 8640, 20)
[7.52s] iter: 285, loss: 0.5940, input dim: (50, 8640, 20)
[5.84s] iter: 286, loss: 0.7546, input dim: (50, 8640, 20)
[7.02s] iter: 287, loss: 0.6501, input dim: (50, 8640, 20)
[5.47s] iter: 288, loss: 0.7828, input dim: (50, 8640, 20)
[6.84s] iter: 289, loss: 0.7412, input dim: (50, 8640, 20)
[7.05s] iter: 290, loss: 0.6875, input dim: (50, 8640, 20)
[6.85s] iter: 291, loss: 0.7350, input dim: (50, 8640, 20)
[6.90s] iter: 292, loss: 0.6409, input dim: (50, 8640, 2

[7.97s] iter: 414, loss: 0.5726, input dim: (50, 8640, 20)
[8.65s] iter: 415, loss: 0.6334, input dim: (50, 8640, 20)
[7.61s] iter: 416, loss: 0.6521, input dim: (50, 8640, 20)
[7.39s] iter: 417, loss: 0.6140, input dim: (50, 8640, 20)
[8.39s] iter: 418, loss: 0.7166, input dim: (50, 8640, 20)
[8.83s] iter: 419, loss: 0.7096, input dim: (50, 8640, 20)
[11.39s] iter: 420, loss: 0.5586, input dim: (50, 8640, 20)
[10.61s] iter: 421, loss: 0.5796, input dim: (50, 8640, 20)
[8.87s] iter: 422, loss: 0.5869, input dim: (50, 8640, 20)
[6.89s] iter: 423, loss: 0.7641, input dim: (50, 8640, 20)
[6.78s] iter: 424, loss: 0.6694, input dim: (50, 8640, 20)
[7.49s] iter: 425, loss: 0.6841, input dim: (50, 8640, 20)
[6.23s] iter: 426, loss: 0.6733, input dim: (50, 8640, 20)
[7.17s] iter: 427, loss: 0.7682, input dim: (50, 8640, 20)
[6.83s] iter: 428, loss: 0.5794, input dim: (50, 8640, 20)
[8.11s] iter: 429, loss: 0.5925, input dim: (50, 8640, 20)
[7.89s] iter: 430, loss: 0.5564, input dim: (50, 8640,

[7.19s] iter: 553, loss: 0.6322, input dim: (50, 8640, 20)
[7.30s] iter: 554, loss: 0.5925, input dim: (50, 8640, 20)
[7.50s] iter: 555, loss: 0.5627, input dim: (50, 8640, 20)
[8.45s] iter: 556, loss: 0.7253, input dim: (50, 8640, 20)
[8.64s] iter: 557, loss: 0.7533, input dim: (50, 8640, 20)
[6.51s] iter: 558, loss: 0.7589, input dim: (50, 8640, 20)
[5.75s] iter: 559, loss: 0.6693, input dim: (50, 8640, 20)
[6.77s] iter: 560, loss: 0.6932, input dim: (50, 8640, 20)
[5.45s] iter: 561, loss: 0.7778, input dim: (50, 8640, 20)
[6.55s] iter: 562, loss: 0.7408, input dim: (50, 8640, 20)
[7.07s] iter: 563, loss: 0.6701, input dim: (50, 8640, 20)
[6.05s] iter: 564, loss: 0.6266, input dim: (50, 8640, 20)
[8.53s] iter: 565, loss: 0.6621, input dim: (50, 8640, 20)
[8.47s] iter: 566, loss: 0.6298, input dim: (50, 8640, 20)
[7.91s] iter: 567, loss: 0.6128, input dim: (50, 8640, 20)
[7.50s] iter: 568, loss: 0.6834, input dim: (50, 8640, 20)
[6.22s] iter: 569, loss: 0.7163, input dim: (50, 8640, 2

...

[7.03s] iter: 1104, loss: 0.7014, input dim: (50, 8640, 20)
[6.71s] iter: 1105, loss: 0.7396, input dim: (50, 8640, 20)
[6.71s] iter: 1106, loss: 0.8571, input dim: (50, 8640, 20)
[5.81s] iter: 1107, loss: 0.6258, input dim: (50, 8640, 20)
[6.85s] iter: 1108, loss: 0.6885, input dim: (50, 8640, 20)
[7.46s] iter: 1109, loss: 0.5713, input dim: (50, 8640, 20)
[9.77s] iter: 1110, loss: 0.7498, input dim: (50, 8640, 20)
[8.82s] iter: 1111, loss: 0.6055, input dim: (50, 8640, 20)
[9.25s] iter: 1112, loss: 0.5124, input dim: (50, 8640, 20)
[8.34s] iter: 1113, loss: 0.6298, input dim: (50, 8640, 20)
[8.52s] iter: 1114, loss: 0.5965, input dim: (50, 8640, 20)
[7.69s] iter: 1115, loss: 0.5460, input dim: (50, 8640, 20)
[7.67s] iter: 1116, loss: 0.7708, input dim: (50, 8640, 20)
[6.82s] iter: 1117, loss: 0.8439, input dim: (50, 8640, 20)
[6.64s] iter: 1118, loss: 0.5955, input dim: (50, 8640, 20)
[6.80s] iter: 1119, loss: 0.6341, input dim: (50, 8640, 20)
[9.31s] iter: 1120, loss: 0.6597, input 

[7.01s] iter: 1240, loss: 0.5833, input dim: (50, 8640, 20)
[7.74s] iter: 1241, loss: 0.5489, input dim: (50, 8640, 20)
[8.28s] iter: 1242, loss: 0.6455, input dim: (50, 8640, 20)
[8.15s] iter: 1243, loss: 0.6169, input dim: (50, 8640, 20)
[7.92s] iter: 1244, loss: 0.6919, input dim: (50, 8640, 20)
[7.24s] iter: 1245, loss: 0.6665, input dim: (50, 8640, 20)
[7.42s] iter: 1246, loss: 0.6404, input dim: (50, 8640, 20)
[8.42s] iter: 1247, loss: 0.5436, input dim: (50, 8640, 20)
[8.04s] iter: 1248, loss: 0.6915, input dim: (50, 8640, 20)
[8.23s] iter: 1249, loss: 0.5051, input dim: (50, 8640, 20)
[10.21s] iter: 1250, loss: 0.5656, input dim: (50, 8640, 20)
[8.19s] iter: 1251, loss: 0.6723, input dim: (50, 8640, 20)
[8.32s] iter: 1252, loss: 0.6038, input dim: (50, 8640, 20)
[11.15s] iter: 1253, loss: 0.5640, input dim: (50, 8640, 20)
[9.63s] iter: 1254, loss: 0.5774, input dim: (50, 8640, 20)
[6.86s] iter: 1255, loss: 0.5733, input dim: (50, 8640, 20)
[7.63s] iter: 1256, loss: 0.5074, inpu

[7.01s] iter: 1377, loss: 0.7125, input dim: (50, 8640, 20)
[6.49s] iter: 1378, loss: 0.5449, input dim: (50, 8640, 20)
[7.45s] iter: 1379, loss: 0.4903, input dim: (50, 8640, 20)
[8.91s] iter: 1380, loss: 0.5893, input dim: (50, 8640, 20)
[9.05s] iter: 1381, loss: 0.6417, input dim: (50, 8640, 20)
[9.39s] iter: 1382, loss: 0.6174, input dim: (50, 8640, 20)
[10.32s] iter: 1383, loss: 0.6599, input dim: (50, 8640, 20)
[9.13s] iter: 1384, loss: 0.6482, input dim: (50, 8640, 20)
[10.31s] iter: 1385, loss: 0.5906, input dim: (50, 8640, 20)
[8.65s] iter: 1386, loss: 0.6093, input dim: (50, 8640, 20)
[7.61s] iter: 1387, loss: 0.6964, input dim: (50, 8640, 20)
[6.46s] iter: 1388, loss: 0.7000, input dim: (50, 8640, 20)
[6.71s] iter: 1389, loss: 0.6839, input dim: (50, 8640, 20)
[7.40s] iter: 1390, loss: 0.5991, input dim: (50, 8640, 20)
[7.41s] iter: 1391, loss: 0.5027, input dim: (50, 8640, 20)
[7.66s] iter: 1392, loss: 0.5318, input dim: (50, 8640, 20)
[8.31s] iter: 1393, loss: 0.8041, inpu

[7.58s] iter: 1513, loss: 0.7969, input dim: (50, 8640, 20)
[9.03s] iter: 1514, loss: 0.6309, input dim: (50, 8640, 20)
[7.49s] iter: 1515, loss: 0.6704, input dim: (50, 8640, 20)
[8.41s] iter: 1516, loss: 0.6353, input dim: (50, 8640, 20)
[7.52s] iter: 1517, loss: 0.6012, input dim: (50, 8640, 20)
[7.70s] iter: 1518, loss: 0.5869, input dim: (50, 8640, 20)
[7.68s] iter: 1519, loss: 0.6461, input dim: (50, 8640, 20)
[8.30s] iter: 1520, loss: 0.6364, input dim: (50, 8640, 20)
[11.79s] iter: 1521, loss: 0.5127, input dim: (50, 8640, 20)
[8.53s] iter: 1522, loss: 0.5603, input dim: (50, 8640, 20)
[8.02s] iter: 1523, loss: 0.4842, input dim: (50, 8640, 20)
[7.43s] iter: 1524, loss: 0.5877, input dim: (50, 8640, 20)
[7.27s] iter: 1525, loss: 0.7933, input dim: (50, 8640, 20)
[7.09s] iter: 1526, loss: 0.7381, input dim: (50, 8640, 20)
[7.26s] iter: 1527, loss: 0.4461, input dim: (50, 8640, 20)
[6.54s] iter: 1528, loss: 0.6617, input dim: (50, 8640, 20)
[7.07s] iter: 1529, loss: 0.6724, input

KeyboardInterrupt: 

## test
model.predict_prob returns prob prediction, used for AUC later

In [3]:
D_config = {"batch_size":50, "shuffle":False, "balance":False, "padding":False, "free_mem":True}
# good model with regularization
model = deep_cnn_lstm_model
model.initial(os.path.join(model_dir, "DeepCNN_LSTM-3000.model"))
m_solver = Solver(num_file_in_mem=4, num_threads=None) # num_threads=None means to use all cores
m_solver.test(model=model, data_dir=data_dir, fnames=test_f, D_config=D_config, use_logits=True)
model.finish()


INFO:tensorflow:Restoring parameters from E:\cs341-project\models\DeepCNN_LSTM-3000.model
model restored.
[INFO] Notice that we are using multiprocessing to load files, so that child processes won't print out on ipython-notebook, which only monitor the parent process. Please check the terminal for more logging info.
batch shape: (29, 8640, 19) loss: 0.5013, acc: 0.9655, AUC: 0.9970
batch shape: (50, 8640, 19) loss: 0.6374, acc: 0.7000, AUC: 0.5714
batch shape: (50, 8640, 19) loss: 0.4875, acc: 0.8600, AUC: 0.5598
batch shape: (50, 8640, 19) loss: 0.4267, acc: 1.0000, AUC: N/A
batch shape: (39, 8640, 19) loss: 0.8901, acc: 0.4103, AUC: 1.0000
batch shape: (50, 8640, 19) loss: 0.4193, acc: 1.0000, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.4610, acc: 0.9600, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.4849, acc: 0.9800, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.6174, acc: 0.8200, AUC: 0.2060
batch shape: (50, 8640, 19) loss: 0.5592, acc: 0.9600, AUC: N/A
batch shape: (50, 8640, 19)

In [3]:
D_config = {"batch_size":50, "shuffle":False, "balance":False, "padding":False, "free_mem":True}
# using good models
model = deep_cnn_lstm_model
model.initial(os.path.join(good_model_dir, "DeepCNN_LSTM-3000.model"))
m_solver = Solver(num_file_in_mem=4, num_threads=None) # num_threads=None means to use all cores
m_solver.test(model=model, data_dir=data_dir, fnames=test_f, D_config=D_config, use_logits=True)
model.finish()


INFO:tensorflow:Restoring parameters from E:\cs341-project\models\good_models\DeepCNN_LSTM-3000.model
model restored.
[INFO] Notice that we are using multiprocessing to load files, so that child processes won't print out on ipython-notebook, which only monitor the parent process. Please check the terminal for more logging info.
batch shape: (50, 8640, 19) loss: 0.4987, acc: 0.9000, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.7435, acc: 0.2600, AUC: 0.6371
batch shape: (11, 8640, 19) loss: 0.1604, acc: 1.0000, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.1739, acc: 0.9000, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.0522, acc: 1.0000, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.0849, acc: 1.0000, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.1578, acc: 0.9800, AUC: N/A
batch shape: (24, 8640, 19) loss: 0.4649, acc: 0.7917, AUC: 0.9034
batch shape: (50, 8640, 19) loss: 0.3918, acc: 1.0000, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.4898, acc: 1.0000, AUC: N/A
batch shape: (50, 8640, 

In [3]:
D_config = {"batch_size":50, "shuffle":False, "balance":False, "padding":False, "free_mem":True}
# new normalization
model = deep_cnn_lstm_model
model.initial(os.path.join(model_dir, "DeepCNN_LSTM-1600.model"))
m_solver = Solver(num_file_in_mem=4, num_threads=None) # num_threads=None means to use all cores
m_solver.test(model=model, data_dir=data_dir, fnames=test_f, D_config=D_config, use_logits=True)
model.finish()

INFO:tensorflow:Restoring parameters from E:\cs341-project\models\DeepCNN_LSTM-1600.model
model restored.
[INFO] Notice that we are using multiprocessing to load files, so that child processes won't print out on ipython-notebook, which only monitor the parent process. Please check the terminal for more logging info.
batch shape: (50, 8640, 19) loss: 0.7680, acc: 0.7000, AUC: 0.6314
batch shape: (50, 8640, 19) loss: 0.4115, acc: 0.7800, AUC: 0.7724
batch shape: (50, 8640, 19) loss: 1.4092, acc: 0.0400, AUC: N/A
batch shape: (39, 8640, 19) loss: 2.1506, acc: 0.1282, AUC: 0.9235
batch shape: (50, 8640, 19) loss: 0.1092, acc: 1.0000, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.0999, acc: 1.0000, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.2569, acc: 0.9200, AUC: N/A
batch shape: (50, 8640, 19) loss: 1.7261, acc: 0.0200, AUC: N/A
batch shape: (24, 8640, 19) loss: 1.7580, acc: 0.2917, AUC: 0.9412
batch shape: (29, 8640, 19) loss: 0.1099, acc: 1.0000, AUC: 1.0000
batch shape: (50, 8640, 19)

In [4]:
D_config = {"batch_size":50, "shuffle":False, "balance":False, "padding":False, "free_mem":True}
# new normalization
model = deep_cnn_lstm_model
model.initial(os.path.join(model_dir, "DeepCNN_LSTM-1600.model"))
m_solver = Solver(num_file_in_mem=4, num_threads=None) # num_threads=None means to use all cores
m_solver.test(model=model, data_dir=data_dir, fnames=train_f, D_config=D_config, use_logits=True)
model.finish()

INFO:tensorflow:Restoring parameters from E:\cs341-project\models\DeepCNN_LSTM-1600.model
model restored.
[INFO] Notice that we are using multiprocessing to load files, so that child processes won't print out on ipython-notebook, which only monitor the parent process. Please check the terminal for more logging info.
batch shape: (50, 8640, 19) loss: 0.1846, acc: 0.9600, AUC: 0.5312
batch shape: (50, 8640, 19) loss: 1.5867, acc: 0.1600, AUC: 0.0000
batch shape: (41, 8640, 19) loss: 1.9561, acc: 0.3415, AUC: 0.6402
batch shape: (50, 8640, 19) loss: 0.1373, acc: 1.0000, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.1349, acc: 1.0000, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.1732, acc: 1.0000, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.5833, acc: 0.8000, AUC: N/A
batch shape: (50, 8640, 19) loss: 1.4168, acc: 0.0600, AUC: N/A
batch shape: (50, 8640, 19) loss: 1.7774, acc: 0.2600, AUC: 0.9189
batch shape: (1, 8640, 19) loss: 0.0622, acc: 1.0000, AUC: N/A
batch shape: (50, 8640, 19) los

batch shape: (8, 8640, 19) loss: 0.4542, acc: 0.8750, AUC: 1.0000
batch shape: (50, 8640, 19) loss: 0.1029, acc: 1.0000, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.1075, acc: 1.0000, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.1094, acc: 1.0000, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.1190, acc: 1.0000, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.1655, acc: 0.9800, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.4518, acc: 0.8400, AUC: N/A
batch shape: (50, 8640, 19) loss: 1.6271, acc: 0.0200, AUC: N/A
batch shape: (50, 8640, 19) loss: 2.4034, acc: 0.0000, AUC: N/A
batch shape: (50, 8640, 19) loss: 2.2956, acc: 0.1400, AUC: 0.6196
batch shape: (17, 8640, 19) loss: 1.6979, acc: 0.4118, AUC: 0.5000
batch shape: (50, 8640, 19) loss: 0.7010, acc: 0.7600, AUC: 0.6568
batch shape: (50, 8640, 19) loss: 0.1161, acc: 1.0000, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.1257, acc: 1.0000, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.1610, acc: 0.9800, AUC: N/A
batch shape: (50, 8640, 19) l

batch shape: (50, 8640, 19) loss: 0.5526, acc: 0.6800, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.8090, acc: 0.4000, AUC: N/A
batch shape: (50, 8640, 19) loss: 1.6269, acc: 0.0400, AUC: N/A
batch shape: (39, 8640, 19) loss: 1.9348, acc: 0.1795, AUC: 0.8438
batch shape: (50, 8640, 19) loss: 0.1148, acc: 1.0000, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.1165, acc: 1.0000, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.1314, acc: 1.0000, AUC: N/A
batch shape: (50, 8640, 19) loss: 0.1715, acc: 1.0000, AUC: N/A
batch shape: (50, 8640, 19) loss: 1.0989, acc: 0.4800, AUC: 0.3945
batch shape: (50, 8640, 19) loss: 2.5768, acc: 0.0200, AUC: N/A
batch shape: (14, 8640, 19) loss: 1.3107, acc: 0.5714, AUC: 0.8367
batch shape: (50, 8640, 19) loss: 0.1197, acc: 1.0000, AUC: N/A
batch shape: (50, 8640, 19) loss: 1.0724, acc: 0.3200, AUC: N/A
batch shape: (35, 8640, 19) loss: 1.4249, acc: 0.4286, AUC: 0.4867
batch shape: (50, 8640, 19) loss: 0.0762, acc: 1.0000, AUC: N/A
batch shape: (50, 8640, 19) 

In [ ]:
D_config = {"batch_size":120, "shuffle":False, "balance":False, "padding":False, "free_mem":True}

model = lstm_model
model.initial(os.path.join(model_dir, "Basic_LSTM-1400.model"))
m_solver = Solver(num_file_in_mem=6, num_threads=None) # num_threads=None means to use all cores
m_solver.test(model=model, data_dir=data_dir, fnames=test_f, D_config=D_config)
model.finish()

# Take a closer look?
* **m_solver.all_y** is the ground truch while **m_solver.all_pred** is the prediction result.

In [ ]:
model.finish()
print(m_solver.all_y)
print(m_solver.all_pred)

In [5]:
print([m_solver.all_y])
b = np.concatenate(m_solver.all_pred)
b

[array([0, 0, 0, ..., 1, 1, 1], dtype=int64)]


array([ 0.3433553 , -0.38856956,  0.32670319, ...,  0.03293468,
        0.25059864, -0.16984977], dtype=float32)